In [4]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/5aa5257608b61e8fcc828e99fbd070d5ca7358e3/mp.py

In [5]:
from glob import glob
from tqdm import tqdm
import pandas as pd
import mp
import numpy as np
from datasets import Audio
import json
import os
import random

chunks = 30
sr = 16000

In [6]:
!mkdir indices
# !rm indices/*

mkdir: cannot create directory ‘indices’: File exists


In [9]:
files = sorted(glob('/home/ubuntu/crawl-youtube/data/*.parquet'))[::-1]
len(files)

2137

In [10]:
def loop(files):
    files, _ = files
    
    audio = Audio(sampling_rate=sr)
    
    for f in tqdm(files):
        filename = os.path.join('indices', os.path.split(f)[1])
        if os.path.exists(filename):
            continue
            
        df = pd.read_parquet(f)
        
        indices = []
        for i in range(len(df)):
            row = df.iloc[i]
            a = audio.decode_example(audio.encode_example(row['filename']))
            l = len(a['array']) // (chunks * sr)
            d = {
                'i': i,
                'l': l,
                'filename': f
            }
            indices.append(d)
        
        with open(filename, 'w') as fopen:
            json.dump(indices, fopen)
            
        del df

In [11]:
r = mp.multiprocessing(files, loop, cores = 30, returned = False)

  0%|          | 0/71 [00:00<?, ?it/s]

100%|██████████| 71/71 [00:00<00:00, 125440.43it/s]

100%|██████████| 71/71 [00:00<00:00, 125917.79it/s]


100%|██████████| 71/71 [00:00<00:00, 136415.75it/s]

100%|██████████| 71/71 [00:00<00:00, 155832.33it/s]

100%|██████████| 71/71 [00:00<00:00, 147496.57it/s]


100%|██████████| 71/71 [00:00<00:00, 167113.12it/s]

100%|██████████| 71/71 [36:01<00:00, 30.45s/it]


In [ ]:
global_indices = {}
start = 0
for indices in r:
    for k, v in indices.items():
        v['start'] = start
        v['end'] = k
        global_indices[start] = v
        start += k

In [ ]:
global_indices